Cài các thư viện 

In [2]:
!pip install transformers
!pip install pytorch-crf
!pip install torchmetrics
import torch
from transformers import AutoModel, AutoTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 9.2 MB/s eta 0:00:00


Dataset : 
- Sử dụng API tokenizer có sẵn 
- chuẩn hóa, vector hóa dữ liệu 

In [ ]:

#API
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
"""
class Build_Vocab():
  def __init__(self, file1, file2):
    f1 = open(file1)
    f2 = open(file2)
    rows1 = f1.readlines()
    rows2 = f2.readlines()
    rows = rows1 + rows2
    self.maxlen = 75
   
    tokenizer = Tokenizer(oov_token = '<oov>')
    tokenizer.fit_on_texts([[i.split()[0]] for i in rows if i != '\n'])
    self.vocab = tokenizer.word_index
    self.vocab_reverse = {}
    for key, value in tokenizer.word_index.items():
      self.vocab_reverse[value] = key
    self.vocab_reverse[0] = ''
  def __len__(self):
    return len(self.vocab)
"""
class Dataset():
  def __init__(self, file, max_len=160):
    f = open(file)
    rows = f.readlines()
    self.maxlen = max_len
    self.data = {
        "sentences" : [],
        "labels" : []
    }
   
    sentence = []
    label = []
    for row in rows:
      if row == '\n':
        self.data["sentences"].append(sentence)
        self.data["labels"].append(label)
       
        sentence = []
        label = []
      else:  
        list_row = row.split() 
        sentence.append(list_row[0])
        label.append(list_row[1])

  def __getitem__(self, i):
    
    sent =  self.data["sentences"][i]
    
    #padding câu 
    sent_pad = pad_sequences([sent], self.maxlen, object, padding='post', value='<pad>')
    sent = " ".join(sent)
    seq = tokenizer.encode(sent)[1:-1]

    seq_pad =  pad_sequences([seq], self.maxlen, padding='post', value=1.0)
    tag = self.data["labels"][i]

    #padding tags
    tags_pad = pad_sequences([tag], self.maxlen, object, padding='post', value='pad')[0]
    tags = []
    tags_number = []
    tags_number_nonPad = []
    for i in range(len(tags_pad)):
      tags.append(One_Hot(tags_pad[i]))
      tags_number.append(normLabel(tags_pad[i]))
  
     
  
    return {
        "sentences": " ".join(sent_pad[0]),
        "sequences": np.array(seq_pad[0]),
        "tags": np.array(tags),
        "tags_number": np.array(tags_number),
       
    }
  def __len__(self):
    return len(self.data["sentences"])
l = ["O", "B-LOCATION", "I-LOCATION", "B-PATIENT_ID", "I-PATIENT_ID", 
       "B-ORGANIZATION", "I-ORGANIZATION", 
       "I-SYMPTOM_AND_DISEASE", "B-SYMPTOM_AND_DISEASE", 
       "B-NAME","I-JOB","B-JOB",
       "B-DATE", "I-DATE", "B-TRANSPORTATION",
       "B-AGE", "B-GENDER", "PAD"]
def normLabel(label):
  try:
    return l.index(label)
    
  except:
    return len(l)
    
def One_Hot(label):
  seq = np.zeros(len(l) + 1)
  try:
    #return l.index(label)
    seq[l.index(label)] = 1
    return seq
  except:
    #return l.index('O')
    seq[len(l)] = 1
    return seq

TAG_SIZE = len(l)
PATH_TRAIN = "/content/drive/MyDrive/Cuongdb/DataNLP/train_word.txt"
PATH_TEST = "/content/drive/MyDrive/Cuongdb/DataNLP/test_word.txt"
dataset = Dataset(PATH_TRAIN)
dataset_val = Dataset(PATH_TEST)
from torch.utils.data import DataLoader 
dataloader = DataLoader(dataset, batch_size = 128)
dataloader_val = DataLoader(dataset_val, batch_size = 128)

MẪU DATASET 

In [4]:
dataset[0]

{'sentences': 'Đồng_thời , bệnh_viện tiếp_tục thực_hiện các biện_pháp phòng_chống dịch_bệnh COVID - 19 theo hướng_dẫn của Bộ Y_tế . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>',
 'sequences': 

# MODEL PRE_TRAIN WITH CRFs

In [ ]:
phobert = AutoModel.from_pretrained("vinai/phobert-base")


Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from torch import nn
from torchcrf import CRF
TAG_SIZE = 19
device = "cuda" if torch.cuda.is_available() else "cpu"
class NERbert(nn.Module):
  def __init__(self, pre_train, n_dim, num_tags):
    super().__init__()
    self.BERT = pre_train
    for child in self.BERT.children():
      for param in child.parameters():
          param.requires_grad = False
    self.fc = nn.Linear(n_dim, num_tags)
    self.crf = CRF(num_tags, batch_first=True)
  def forward(self, x, tags=None, pad_id=1):
    x_masks = self.make_bert_mask(x, pad_id)  # (B, L)
    
    features = self.BERT(x, attention_mask=x_masks)[0] #(B, L, N_dim) , (B, N_dim)

    #(B, L, N) = feature.size()
    output = self.fc(features) #(B, L, N_tag)
 
    #print(output.shape[:2], tags.shape, x_masks.shape)
    if tags is not None:
      log_likelihood = self.crf(output, tags, mask=x_masks.bool(), reduction='mean')
    else:
      log_likelihood = None
    sequence_of_tags = self.crf.decode(output, mask=x_masks.bool())
    #return log_likelihood, sequence_of_tags
    
    sequence_of_tags = pad_sequences(sequence_of_tags, maxlen=x.size()[1], padding="post", value = 18)
    if tags is not None :
      return torch.Tensor(log_likelihood), torch.Tensor(sequence_of_tags)  # (), (B, L)
    else :
      return torch.Tensor(sequence_of_tags)
    
  def make_bert_mask(self, x, pad_id):
    bert_masks = (x != pad_id).float()  # (B, L)
    return torch.Tensor(bert_masks)
model = NERbert(phobert, 768, TAG_SIZE)
model = model.to(device)
"""
input = torch.Tensor([dataset[0]["sequences"]]).to(torch.int64)
tags = torch.Tensor([dataset[0]["tags_number"]]).to(torch.int64)
loss,output = model(input, tags)

"""
optim = torch.optim.AdamW(model.parameters(), lr=0.01)


In [ ]:
print(model)

NERbert(
  (BERT): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=

In [ ]:
from transformers.modeling_tf_utils import input_processing
from torchmetrics import F1Score
f1 = F1Score(task="multiclass", num_classes=len(l))
f1.to(device)
def f1_function(o, t, len = 160):
  o1_pad = pad_sequences(o, maxlen = len, padding = "post", value = 18)
  p1 = f1(o1_pad, t)
  print(o1_pad)
  o2_pad = pad_sequences(o, maxlen = len, padding = "post", value = 19)
  p2 = f1(o2_pad, t)
  print(o2_pad)
  return p2 / (1-p1-p2)
def train(model, device, optim, dataloader):
    total_loss = 0
    f1_score = 0
    id_pad = 18
    model.train()
    for i, batch in enumerate(dataloader):
      inputs = batch["sequences"].to(device)
      tags = batch["tags_number"].to(device)
   
      log_loss, outputs = model(inputs, tags)
      outputs = outputs.to(device)
      f1_score += f1(outputs, tags)
      loss = -1 * log_loss
      model.zero_grad()
      optim.zero_grad()
      loss.backward()
      optim.step()   
      total_loss += loss.detach().item()
    return total_loss/len(dataloader), f1_score/(len(dataloader)), model
def validation(model, device, dataloader):
  total_loss_val = 0
  model.eval()
  f1_score = 0
  with torch.no_grad():
    for i, batch in enumerate(dataloader):
      inputs = batch["sequences"].to(device)
      tags = batch["tags_number"].to(device)
 
      log_loss, outputs = model(inputs, tags)
      outputs = outputs.to(device)
      f1_score += f1(outputs, tags)
      loss_val = -1 * log_loss
      total_loss_val += loss_val.detach().item()
  return total_loss_val/len(dataloader), f1_score/(len(dataloader))
            

In [ ]:
def train_val(model, device, epochs, optim, dataloader, dataloader_val):
  best_loss = np.Inf 
  PATH = "/content/drive/MyDrive/Cuongdb/NerBer2.pth"
  for epoch in range(epochs):
    print("#"*50 + f" Epoch: {epoch} " + "#"*50)
    loss_train, f1, model_trained = train(model, device, optim, dataloader)
    loss_val, f1_val = validation(model_trained, device, dataloader_val)
    if best_loss > loss_val:
      best_loss = loss_val
      torch.save(model_trained, PATH)
    print(f"loss : {loss_train}  -  f1 : {f1}  -   loss_val : {loss_val}   -  f1_val = {f1_val}")
  return model_trained
train_val(model, device, 200, optim, dataloader, dataloader_val)

In [ ]:
bermodel = NERbert(phobert, 768, TAG_SIZE)
bermodel = torch.load("/content/drive/MyDrive/Cuongdb/NerBer.pth")

In [ ]:
ids = 2
input = torch.tensor([dataset_val[ids]["sequences"]]).to(device)
tag = torch.zeros((1,160)).to(torch.int64).to(device) #torch.tensor([dataset_val[ids]["tags_number"]]).to(device)
_, output = bermodel(input, tag)
for i in range(len(output[0])):
  id = int(input[0][i].cpu().detach().numpy())
  id_o = int(output[0][i].cpu().detach().numpy())
  if id != 1:
    print(tokenizer.convert_ids_to_tokens(id), l[id_o])
    

Theo O
đó O
, O
Sở B-ORGANIZATION
Y_tế I-ORGANIZATION
Bình_Thuận I-ORGANIZATION
cho O
biết O
sau O
khi O
xác_định O
bệnh_nhân O
số O
34 B-PATIENT_ID
( O
nữ_giới O
51 O
tuổi O
, O
từ O
Mỹ B-LOCATION
về O
Việt_Nam O
ngày O
29 B-DATE
- I-DATE
2 I-DATE
có O
quá_cảnh O
Qatar O
) O
, O
Trung_tâm B-ORGANIZATION
Kiểm_soát I-ORGANIZATION
bệnh_tật I-ORGANIZATION
Bình_Thuận I-ORGANIZATION
đã O
điều_tra O
dịch_tễ O
, O
khoanh O
vùng O
, O
khử O
khuẩn O
, O
tiến_hành O
cách_ly O
người O
liên_quan O
đến O
ca O
bệnh O
số O
34 B-PATIENT_ID
. O
